<a href="https://colab.research.google.com/github/Manishkumarsingh41/project/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain_community langchain_chroma langchain_openai langchain_core unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.9/437.9 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from langchain_community.document_loaders import UnstructuredURLLoader
urls = ['https://www.rnsit.ac.in/']
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [ ]:
# from langchain_community.document_loaders import PyPDFLoader  # or use another loader as needed

# # Example: load a PDF file
# loader = PyPDFLoader("/content/Manish_kumar_singh_Resume format.pdf")
# data = loader.load()

In [2]:
data

[Document(metadata={'source': 'https://www.rnsit.ac.in/'}, page_content="Student/Parent Portal\n\nEvents\n\nEvent Calendar News & Events\n\nHall of fame\n\nBE Honors VTU Ranks BOS\n\nInnovative TLP\n\nAccrediations\n\nNAAC NIRF NBA\n\nAutonomous\n\nRegulations - UG Regulations - MCA Regulations - MBA Syllabus - First Year (2024 Scheme)\n\nFee Payment\n\nDocuments & Disclosures\n\nCommittees AICTE VTU Service Rules Faculty Details Financial Audit Reports\n\nFee Structure\n\nManagement Quota Fee 2024 COMEDK Fee Structure 2024\n\nAdmissions - 2025 Open Now\n\nE104\n\nCOMEDK CODE\n\nE118\n\nCET CODE\n\nCoE Notifications\n\nCirculars\n\n28APR 2025\n\nPhotocopy & Revaluation Application for PG\n\nDownload Now\n\n25APR 2025\n\nNotification for Revaluation and Photocopy PG\n\nDownload Now\n\n19APR 2025\n\nMaintaining Minimum Attendance and SEE requirements\n\nDownload Now\n\n03APR 2025\n\nAutonomous Calendar of Events\n\nDownload Now\n\nView All\n\n21APR 2025\n\nPG Calender of events sem 2\n\n

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  568


In [4]:
docs[0]

Document(metadata={'source': 'https://www.rnsit.ac.in/'}, page_content='Student/Parent Portal\n\nEvents\n\nEvent Calendar News & Events\n\nHall of fame\n\nBE Honors VTU Ranks BOS\n\nInnovative TLP\n\nAccrediations\n\nNAAC NIRF NBA\n\nAutonomous')

In [5]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI

vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings(), )

In [7]:
vectorstore

In [8]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What Is Retrieval-Augmented Generation ?")

In [9]:
len(retrieved_docs)

6

In [10]:
retrieved_docs[0]

Document(id='218b57d0-147e-488f-8309-27f2f664ccd6', metadata={'source': 'https://www.rnsit.ac.in/'}, page_content='Promoting excellence in Teaching, Research and Consultancy that galvanizes academic consciousness among Faculty and Students')

In [11]:
print(retrieved_docs[0].page_content)

Promoting excellence in Teaching, Research and Consultancy that galvanizes academic consciousness among Faculty and Students


In [12]:
llm = OpenAI(temperature=0.4, max_tokens=500)

In [13]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [14]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [15]:
def ask_question(question):
    response = rag_chain.invoke({"input": question})
    return response["answer"]

In [16]:
responce = ask_question ("tell me about Campus Life ?")
print(responce)



The campus life at this university is vibrant and diverse, with a mix of academic and extracurricular activities. Students have access to top-notch infrastructure, including lush green spaces and modern facilities. The faculty and students have a good rapport, creating a lively and engaging atmosphere. The university also offers top placement opportunities and various committees, scholarships, and career development programs for students.


In [17]:
response = rag_chain.invoke({"input": "tell me about Campus Life ?"})
print(response["answer"])



The campus life at this university is very vibrant and diverse. There are various tech events, cultural activities, and other extracurricular activities that make it a fun and engaging experience for students. The faculty and students have a good rapport, making the campus environment lively.


In [18]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 128.9 MB/s eta 0:00:00


In [19]:
import gradio as gr

interface = gr.Interface(
    fn=ask_question,
    inputs="text",
    outputs="text",
    title="RAG Chatbot",
    description="Ask a question and get context-aware answers from your knowledge base."
)

interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ee04b094db57d339c4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
